# AML - 2025 : Feather in Focus - The Baseline

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
from google.colab import drive

In [2]:
# 1. SETUP: Mount Google Drive
# ---------------------------------------------------------
drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/AML2025"

DATA_PATH = os.path.join(BASE_PATH, "Dataset")

Mounted at /content/drive
